In [1]:
import pandas as pd

In [2]:
# Utilities for plotting; not part of week 2 content
# All plotting can also be done without this code block
import matplotlib.pyplot as plt

# Render our plots inline
%matplotlib inline

# Make the graphs a bit prettier, and bigger
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)

# Weather data
We consider a 2012 weather case data from Canada.

In [3]:
weather = pd.read_csv('../data/weather_2012.csv')
weather

,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,2012-01-01 03:00:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,2012-01-01 04:00:00,-1.5,-3.3,88,7,4.8,101.23,Fog
...,...,...,...,...,...,...,...,...
8779,2012-12-31 19:00:00,0.1,-2.7,81,30,9.7,100.13,Snow
8780,2012-12-31 20:00:00,0.2,-2.4,83,24,9.7,100.03,Snow
8781,2012-12-31 21:00:00,-0.5,-1.5,93,28,4.8,99.95,Snow
8782,2012-12-31 22:00:00,-0.2,-1.8,89,28,9.7,99.91,Snow


The `Date/Time` column is currently not using datetime objects like we did in the bike paths notebook. We can easily convert it to datetime as follows:

In [4]:
weather['Date/Time'] = pd.to_datetime(weather['Date/Time'])
weather['Date/Time']

0      2012-01-01 00:00:00
1      2012-01-01 01:00:00
2      2012-01-01 02:00:00
3      2012-01-01 03:00:00
4      2012-01-01 04:00:00
               ...        
8779   2012-12-31 19:00:00
8780   2012-12-31 20:00:00
8781   2012-12-31 21:00:00
8782   2012-12-31 22:00:00
8783   2012-12-31 23:00:00
Name: Date/Time, Length: 8784, dtype: datetime64[ns]

Now we can use `.hour`, `.month` attributes etc. to extract the time data.

Okay, so what if we want the data for the whole year? Ideally the API would just let us download that, but I couldn't figure out a way to do that.

First, let's put our work from above into a function that gets the weather for a given month. 

I noticed that there's an irritating bug where when I ask for January, it gives me data for the previous year, so we'll fix that too. [no, really. You can check =)]

In [5]:
def download_weather_month(year, month):
    if month == 1:
        year += 1
    url = url_template.format(year=year, month=month)
    weather_data = pd.read_csv(url, skiprows=15, index_col='Date/Time', parse_dates=True, header=True)
    weather_data = weather_data.dropna(axis=1)
    weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
    weather_data = weather_data.drop(['Year', 'Day', 'Month', 'Time', 'Data Quality'], axis=1)
    return weather_data

In [6]:
# Add hour column
weather['Hour'] = weather['Date/Time'].apply(lambda dt: dt.hour)

# Add month column
weather['Month'] = weather['Date/Time'].apply(lambda dt: dt.month)

weather

,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather,Hour,Month
0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog,0,1
1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog,1,1
2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog",2,1
3,2012-01-01 03:00:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog",3,1
4,2012-01-01 04:00:00,-1.5,-3.3,88,7,4.8,101.23,Fog,4,1
...,...,...,...,...,...,...,...,...,...,...
8779,2012-12-31 19:00:00,0.1,-2.7,81,30,9.7,100.13,Snow,19,12
8780,2012-12-31 20:00:00,0.2,-2.4,83,24,9.7,100.03,Snow,20,12
8781,2012-12-31 21:00:00,-0.5,-1.5,93,28,4.8,99.95,Snow,21,12
8782,2012-12-31 22:00:00,-0.2,-1.8,89,28,9.7,99.91,Snow,22,12


# Coldest month

We can test that this function does the right thing:

Now we can use aggregate to find the month that's coldest on average.

In [7]:
import numpy as np
avgmonth = weather.groupby('Month')['Temp (C)'].aggregate(np.mean)
avgmonth

Month
1     -7.371505
2     -4.225000
3      3.121237
4      7.009306
5     16.237769
6     20.134028
7     22.790054
8     22.279301
9     16.484444
10    10.954973
11     0.931389
12    -3.306317
Name: Temp (C), dtype: float64

In [8]:
avgmonth.idxmin()

1

January is the coldest month. What about the warmest month?

In [9]:
avgmonth.idxmax()

7

July is the coldest month.

# Temperature differences

We're interested in the biggest decrease in temperature between two consecutive hours. There's a very easy way to calculate the difference using `diff`:

In [10]:
# first sort the values, diff takes differences between two conseuctive rows
tempchange = weather.sort_values('Date/Time')['Temp (C)'].diff()
weather['tempchange'] = tempchange
weather

,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather,Hour,Month,tempchange
0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog,0,1,NaN
1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog,1,1,0.0
2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog",2,1,0.0
3,2012-01-01 03:00:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog",3,1,0.3
4,2012-01-01 04:00:00,-1.5,-3.3,88,7,4.8,101.23,Fog,4,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8779,2012-12-31 19:00:00,0.1,-2.7,81,30,9.7,100.13,Snow,19,12,1.4
8780,2012-12-31 20:00:00,0.2,-2.4,83,24,9.7,100.03,Snow,20,12,0.1
8781,2012-12-31 21:00:00,-0.5,-1.5,93,28,4.8,99.95,Snow,21,12,-0.7
8782,2012-12-31 22:00:00,-0.2,-1.8,89,28,9.7,99.91,Snow,22,12,0.3


In [11]:
weather.loc[weather['tempchange'].idxmin()]

Date/Time             2012-07-23 17:00:00
Temp (C)                             19.4
Dew Point Temp (C)                   18.2
Rel Hum (%)                            93
Wind Spd (km/h)                         4
Visibility (km)                      24.1
Stn Press (kPa)                    100.24
Weather                Thunderstorms,Rain
Hour                                   17
Month                                   7
tempchange                          -10.5
Name: 4913, dtype: object

The largest drop of temperature was 10.5 degrees. This is quite a lot. What are the values on average per hour?

In [12]:
download_weather_month(2012, 1)[:5]

NameError: name 'url_template' is not defined

In [ ]:
weather.groupby('Hour')['tempchange'].aggregate(np.mean)

Now we can get all the months at once. This will take a little while to run.

In [ ]:
data_by_month = [download_weather_month(2012, i) for i in range(1, 13)]

Once we have this, it's easy to concatenate all the dataframes together into one big dataframe using `pd.concat`. And now we have the whole year's data!

In [ ]:
weather_2012 = pd.concat(data_by_month)
weather_2012